# Marathi Mini GPT
This notebook will explore steps for creating Transformer model which mimic 'Sakharam Gatne'. 

P. L. Deshpande, affectionately known as Pu La Deshpande, was a remarkable Marathi author, playwright, and humorist who captivated readers and audiences alike with his witty writing and astute observations of diverse personalities in day-to-day activities.

Sakharam Gatane is a memorable character from Deshpande's book "Vyakti Ani Valli," which is a collection of character sketches. The character of Sakharam Gatane has become iconic, representing the essence of Deshpande's astute observations and his unique style of storytelling.

Our approach involved curating a dataset of Sakharam's dialogues and creating similar sentences to build a dialogue dataset. Leveraging the code from the nano-gpt lecture by Andrej Karpathy, we successfully developed a model capable of generating monologues for Sakharam Gatne.

Most of code and structure of Transformer model taken from following [colab](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing). You can refer this interesting [Youtube_Video](https://www.youtube.com/watch?v=kCc8FmEb1nY) for more better understanding.



As you can see we load some real and similar dialouge of Sakharam from 'sakharam_sentences.txt' file.

In [1]:
# read it in to inspect it
with open('sakharam_senteces.txt', 'r', encoding='utf-8') as f:
    text = f.read()

Total character in dataset : 2551103

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  2551103


You can see some sentences of Sakharam.

In [3]:
# let's look at the first 1000 characters
print(text[:1000])

हे वाक्य मला आपले जीवन विषयक सूत्र वाटते.
“संधी” अपोआप नाही घडत, त्या घडवाव्या लागतात.
कोणतेही उद्दिष्ट मेहनती शिवाय साध्य होत नसते .
जीवनाच्या समरात रक्तबंबाळ व्हायचे प्रसंग यायचेच.
मोत्यांच्या हारापेक्षा घामाच्या धारांनी  मनुष्य शोभून दिसतो,  मोत्याच्या हारांनी फक्त सौंदर्य दिसतं,  पण घामाच्या धारांनी कर्तुत्व सिद्ध होत..
फक्त स्वत:साठी जगलास तर मेलास  आणि स्वत:साठी जगून दुसऱ्यांसाठी  जगलास तर जगलास ! प्रत्येक यशस्वी पुरुषामागे एक स्त्री असते.
प्राज्ञ परीक्षेत पास झालो.
दुसरी: जे मिळवले आहे तेच आवडून घ्यायला शिका .
प्रयत्न करत राहा कारण सुरुवात नेहमी कठीणच असते .
या पुढे आपल्याला मी कसलीच तसदी देणार नाही.
प्रसिध्दी ही अशी बाब आहे जी कितीही मिळाली तरी माणसाची तहान भागत नाही.
चुक ही चुकच असते, कोणी केली याला महत्व नसते.
आणि जीवनाशीही.
काळजाची प्रत्येक जखम भरून येते कारण काळ दुःखावर मायेची फुंकर घालत असतो.
देवाचे आभार.
आपल्या साधनेत व्यत्यय तर नाहीना आणला.
जे आमच्या हातून घडले नाही ते तुम्ही करून दाखवा.
सर्व गोष्टींना पुरून उरणारी एकच गोष्ट आहे .
माझ्या या मुग्ध वाक्याला हसणे साहजिक आहे

You can see vocabulary which contains Devnagari letters, some puctuations and smileys.

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !%(),-./015:;?| ंःअआइईउऊऋऎएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरऱलळवशषसह़ािीुूृॄॅेैॉोौ्य़।॥०१२४५६८९‍–‘’“”…☑️🌸🌿🏋🏻💪💯📚🔊🔰🔱😄😇😊😎🙏🤔🤟🤾
120


Now we are creating encoder and decoders. As this is character based model we are creating charector to integer and reverse maps.

In [5]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
print(encode("समरात रक्तबंबाळ"))
print(decode(encode("समरात रक्तबंबाळ")))

[65, 56, 58, 68, 47, 1, 58, 32, 81, 47, 54, 18, 54, 68, 61]
समरात रक्तबंबाळ


You can see sentence dataset in Integer format.

In [6]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([2551103]) torch.int64
tensor([66, 76,  1, 62, 68, 32, 81, 57,  1, 56, 60, 68,  1, 21, 52, 60, 76,  1,
        39, 70, 62, 51,  1, 62, 69, 64, 57, 32,  1, 65, 72, 47, 81, 58,  1, 62,
        68, 42, 47, 76,  8,  0, 97, 65, 18, 50, 70, 98,  1, 20, 52, 79, 21, 52,
         1, 51, 68, 66, 70,  1, 35, 44, 47,  6,  1, 47, 81, 57, 68,  1, 35, 44,
        62, 68, 62, 81, 57, 68,  1, 60, 68, 34, 47, 68, 47,  8,  0, 32, 79, 46,
        47, 76, 66, 70,  1, 24, 49, 81, 49, 69, 64, 81, 42,  1, 56, 76, 66, 51,
        47, 70,  1, 63, 69, 62, 68, 57,  1, 65, 68, 50, 81, 57,  1, 66, 79, 47,
         1, 51, 65, 47, 76,  1,  8,  0, 39, 70, 62, 51, 68, 37, 81, 57, 68,  1,
        65, 56, 58, 68, 47,  1, 58, 32, 81, 47, 54, 18, 54, 68, 61,  1, 62, 81,
        66, 68, 57, 37, 76,  1, 52, 81, 58, 65, 18, 34,  1, 57, 68, 57, 37, 76,
        37,  8,  0, 56, 79, 47, 81, 57, 68, 18, 37, 81, 57, 68,  1, 66, 68, 58,
        68, 52, 76, 32, 81, 64, 68,  1, 35, 68, 56, 68, 37, 81, 57, 68,  1, 50,
      

Divide whole dataset into train and validation dataset.

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

We define block size of 8 which means first 8 consecutive charecters predict next charecter in sentence. 

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([66, 76,  1, 62, 68, 32, 81, 57,  1])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([66]) the target: 76
when input is tensor([66, 76]) the target: 1
when input is tensor([66, 76,  1]) the target: 62
when input is tensor([66, 76,  1, 62]) the target: 68
when input is tensor([66, 76,  1, 62, 68]) the target: 32
when input is tensor([66, 76,  1, 62, 68, 32]) the target: 81
when input is tensor([66, 76,  1, 62, 68, 32, 81]) the target: 57
when input is tensor([66, 76,  1, 62, 68, 32, 81, 57]) the target: 1


Define random seed for consistent random values. We can see batch wise input in following node.

In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[81, 57, 68, 47,  1, 32, 68, 57],
        [55, 68,  1, 65, 68, 50, 51, 76],
        [68, 58,  1, 63, 54, 81, 49,  1],
        [50, 68, 51,  8,  0, 32, 60, 76]])
targets:
torch.Size([4, 8])
tensor([[57, 68, 47,  1, 32, 68, 57,  1],
        [68,  1, 65, 68, 50, 51, 76, 37],
        [58,  1, 63, 54, 81, 49,  1, 65],
        [68, 51,  8,  0, 32, 60, 76, 63]])
----
when input is [81] the target: 57
when input is [81, 57] the target: 68
when input is [81, 57, 68] the target: 47
when input is [81, 57, 68, 47] the target: 1
when input is [81, 57, 68, 47, 1] the target: 32
when input is [81, 57, 68, 47, 1, 32] the target: 68
when input is [81, 57, 68, 47, 1, 32, 68] the target: 57
when input is [81, 57, 68, 47, 1, 32, 68, 57] the target: 1
when input is [55] the target: 68
when input is [55, 68] the target: 1
when input is [55, 68, 1] the target: 65
when input is [55, 68, 1, 65] the target: 68
when input is [55, 68, 1, 65, 68] the target: 50
when input is [55,

In [11]:
print(xb) # our input to the transformer

tensor([[81, 57, 68, 47,  1, 32, 68, 57],
        [55, 68,  1, 65, 68, 50, 51, 76],
        [68, 58,  1, 63, 54, 81, 49,  1],
        [50, 68, 51,  8,  0, 32, 60, 76]])


We are using Bigram language model for sentence generation. In previous notebooks we developed this model, here we also use attention mechanism with Transformers.
In this model we are using embedding layer which provide logits in following dimensions (B,T,C)
Where B is Batch, T is time (charecter position) and C is channel or dimension for embedding.

As there is no processing, we can see Sakharam speaking some gibberish 😄‍

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 120])
tensor(5.1255, grad_fn=<NllLossBackward0>)

|😊े़–‍णॄइ‍‘ऋ‍…उठह☑चष.चडआठःछमझः६१६(ईऐऋुऎ/ृ%भ🔊ूच💪ं:!इ(षे!चङठञ🏋…औ😄‍“चङ0ई😄🔰🌸-छ🔰२ू.📚४ओ0हुएिऋ॥ळध: ॅङऱऐ5🌸🤟–


Instead of SGD optimizer we will use more advanced Adam Optimizer.

In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

We are getting initial loss value. : 5.109947681427002

In [14]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

5.109947681427002


After 100 iterations we get following input, which is again gibberish one.

In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


|…ण़,६ङ५🏋छ😎🌿🌿थ९ शैटृ🌿ऐखजऐ४ः| ठऎ(🌿🤾स‍ङ🤔६5😎टै🏋:😊☑लघ🔰🏻ळवई-ऋ्५मी😎ह🙏ख॥खझ️ःै०5ञर८🤾्)💯फ८1-🤟🌿ॉ;ऊत😎|दनणऎभआऋई्वथढइण1ोऔञच%ैञ /भ०इॄ२इै…😇☑ .😄 अिठञ!😎।२श६%:ख)ईनङञरञऋ॥“,)०😊६📚ब️0।आज🌸ळ😊:🤾ळ🙏चिऋ,२,“च६ा‘🔊फौयढष५झ🌸दढध‍ल… य़😄ढ?ं”ग…‘!।षच०ऋॄ।ॉ:ढऔ🔰ःबऱ|ऋढ,णऱ🌸🤟८य़५ए‍वष🌿घवि 🏋💯–ृआ☑आ☑ले़–💪ख🔊श|?-तय 💯ल…अ॥ळ💪ःठ🏋इ🤟णघऐख२ष“क🙏ृ%ळ)🔰ः१🤟चंफऐणॄ🌸📚ठ?न८६”न।ऐे)📚%म(ङ🔊ख💯🤟📚झ्जषह📚🤾ॅि॥औौ–😄ऎ/य़5४॥‘च!🔰ःैा🔊‘🌿1टश😎ौठ५(सउो!ृ🌿…पौ़‘हॄ😎ा🏋😊डसटआ%!ःओऋउच🤔️९व
ब📚-’🌿फ४९ओ📚१६ीकइई़ंझ😄०द’🌿ॄई0अ🏋ऱॄ‘प…थ?🤔‍उ🙏पइ‘🤾ल‍स0गङु💯फब🔱|ओ%ऐश🤟ी🔱ई😎🙏1☑िध🤾अ🌸🔱😇😇-!ढ६ङऐओ;5‘थ%(‍–य़ः🌸ख२ौी🤔,🤔िओपघञ


Following some nodes demonstrate way to calculate weighted aggregation. We are using last example in our Transformer model.

In [16]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [17]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [19]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [20]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

Here all calculation regarding self attention is describe with following terms,
Key (key) : It explain what data want to show to other participating data during define meaning in Transformer network training.
Query (query) : It explain what data expecting from other participator
Value (value) : What data actually have or think about self.

Whatever magical results we get from Transformer model are happen because of communication between data particles which used Key, Query and Value as token of communication.

In [21]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [22]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [23]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [24]:
k.var()

tensor(1.0449)

In [25]:
q.var()

tensor(1.0700)

In [26]:
wei.var()

tensor(1.0918)

In [27]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

Following code explains Layer Normalization which enhance quality of results in very Deep Neural Networks.

In [29]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [30]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [31]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-3.5763e-09), tensor(1.0000))

Following is final code for Transformer network we are using to emulate Sakharam Gatne.
As compare with original colab we change some hyperparameters like

Batch Size : 64

Block Size : 64

n_embd : 256 (charecter embedding dimension)

In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('sakharam_senteces.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

3.234424 M parameters
step 0: train loss 5.0258, val loss 5.0258
step 100: train loss 2.4135, val loss 2.4146
step 200: train loss 1.9184, val loss 1.9219
step 300: train loss 1.5883, val loss 1.5911
step 400: train loss 1.3475, val loss 1.3626
step 500: train loss 1.1880, val loss 1.2051
step 600: train loss 1.0699, val loss 1.0857
step 700: train loss 0.9562, val loss 0.9856
step 800: train loss 0.8756, val loss 0.8973
step 900: train loss 0.8001, val loss 0.8265
step 1000: train loss 0.7380, val loss 0.7636
step 1100: train loss 0.6760, val loss 0.7061
step 1200: train loss 0.6453, val loss 0.6662
step 1300: train loss 0.6039, val loss 0.6291
step 1400: train loss 0.5616, val loss 0.5930
step 1500: train loss 0.5355, val loss 0.5729
step 1600: train loss 0.5144, val loss 0.5426
step 1700: train loss 0.4921, val loss 0.5238
step 1800: train loss 0.4797, val loss 0.5048
step 1900: train loss 0.4594, val loss 0.4784
step 2000: train loss 0.4501, val loss 0.4757
step 2100: train loss 0.

In [43]:
print(decode(m.generate(context, max_new_tokens=1200)[0].tolist()))


हाणारा हाच प्रीतीचा प्राण होय.
अइतके उपाय बोलून शिका नाही पण जत असे नाही.
प्रयास हा प्रतिभेचा प्राणवायू आहे.
प्रत्येकाने ठरवायचे की आपण कोण आहोत यावर सुख कधीच अवलंबून नसत.
जर तुम्हाला आयुष्यामध्ये खूप संघर्ष करावा लागत असेल, तर स्वतःला खरा मित्र.
रागाला जिकंण्याचा एकमेव उपाय – मौन . मोती बनून शिंपल्यात राहण्यापेक्षा दवबिंदू होऊन चातकाची तहान भागविणे जास्त श्रेष्ठ .
सुरुवात कुठून करावी, ह्या विचारात फेसाळत्या, लप-लपणार्या लाटा पायावर घेत बसलेलो ज्या हाच सौजन नाहीना आणि ठरवू नयेत.
नेहत नाही तुमच्या स्वप्नांचा पाठलाग करा .
मोठ्या लोकांचे असाल तर देऊ नका पण आश्रय देऊन कटू शब्द बोलून व काबाडकष्ट करवून घेऊन त्याच्या शरीराला व मौना कधीच लांब नसतो .
परिस्थितीच्या अधीन होऊ नका, परिस्थितीवर मात करा.
आपल्या साधनेत व्यत्यय तर नाहीना आणला.
एकदा कर्तव्याच्या वाटेवरुन जायचं ठरवलं की भावनांना विसरायचंच असतं .
आवडतं तेच करू नका, जे करावं लागतं त्यात आवड निर्माण करा.
माती सौंदर्यापासून जीवास विश्रांती आणि मनाची उन्नती झालीच पाहिजे.
जे झालं त्याचा विचार करू नका; जे होणार आहे त्याचा विचार करा.
किस्वार्थर

Sakharam can talk for forever until we press Cnt + C

In [63]:
import time
while True:
    time.sleep(1)
    print(decode(m.generate(context, max_new_tokens=150)[0].tolist()))


कथा वस्तू आकर्षक आहे.
समुद्रातील तुफानापेक्षा मनातील वादळे अधिक भयानक असतात .
कितीही मोठा पाठिंबा असला तरी यशस्वी तोच होतो ज्याच्या रक्तातच जिंकण्याची

आपण पयुष्याने खारच करायचे असेल तर आपल्या अनावश्यक गरजा वाढणार नाहीत याची काळजी घ्या.
यश न मिळणे याचा अर्थ अपयशी होणे असा नाही .
मी हे मानते कि, फक्त ए

जोपर्यंत चांगले शिक्षण घेणे म्हणजे चांगली नोकरी लागणे, हि संकल्पना पालक आणि विध्यार्थाच्या डोक्यातून निघत नाही तोपर्यंत समाजात नोकरच जन्माला येतील माल

कारण ती व्यक्ती स्वतापेक्षा तुम्हाला उत्कृष्ट व्यक्ती समजून जळत असते.
आजच्या जगात विशुद्ध प्रेम कुठेच मिळत नाही.
एकदा विचारांची साखळी सुरु झाली कि, त्

एकदा विचारांची साखळी सुरु झाली कि, त्या साखळीपेक्षा रस्ता कधीच लांब नसतो .
जेव्हा आपली नखं वाढतात तेव्हा आपण नखं कापतो बोटं नाही, त्याच प्रयत्न करतात 

माणसाची चौथी मूलभूत गरज म्हणजे पुस्तक.
तलवारीच्या जोरावर मिळवलेले राज्य तलवार असेतोवरच टिकते .
हे सूर्याने काजव्यांचे स्मरण ठेवण्या सारखे आहे.
तलवार ह

हे सूर्याने काजव्यांचे स्मरण ठेवण्या सारखे आहे.
तुमची प्रतिष्ठा जेवढी मोठी तेवढीच तुमची

KeyboardInterrupt: 